# Work In Progress (WIP)

# 2. Quantum Logic Gates

In [1]:
#  Ensures you have dependencies installed in your Kaggle Notebooks environment
import os

iskaggle = os.environ.get("KAGGLE_KERNEL_RUN_TYPE", "")

if iskaggle:
    !pip install -U plotly

In [2]:
import numpy as np

## What are quantum logic gates?

In [notebook 1](https://github.com/CarloLepelaars/q4p/blob/main/nbs/1-the-qubit.ipynb) we have firmly established the idea foundation of quantum computing as a state evolving through a series of quantum logic gates. We have also discussed one of the simplest meaningful logic gates called the X (NOT) gate. Here we will discuss the foundation of these gates and the most common implementations. We will start with a single qubit and then move on to multi-qubit gates.


Recall that a quantum state for a single qubit is a vector with 2 complex numbers. Almost all quantum algorithms start with the zero state:

$$
\begin{bmatrix}
1 \\
0
\end{bmatrix} = |0\rangle
$$

In [3]:
zero_state = np.array([1, 0], dtype=complex)
zero_state

array([1.+0.j, 0.+0.j])

## Pauli Gates

The X (NOT) gate discussed in notebook 1is one of the fundamental "Pauli Gates", which allow us to rotate the state around 3 axes. The have a special significance in quantum computing as they are all "observables", which means they can be used for measuring a qubit. Recall that in most quantum algorithms we measure in the "Z-basis". It should also be noted that we can have an "Identity gate (I)", which does not change the state. Even though this sounds trivial, the identity gate becomes important in a multi-qubit system. The Pauli Gates and Identity gate can be described as matrices:

$$
X = \sigma_x = \begin{bmatrix}
0 & 1 \\
1 & 0
\end{bmatrix},
\quad
Y = \sigma_y = \begin{bmatrix}
0 & -i \\
i & 0
\end{bmatrix},
\quad
Z = \sigma_z = \begin{bmatrix}
1 & 0 \\
0 & -1
\end{bmatrix},
\quad
I = \begin{bmatrix}
1 & 0 \\
0 & 1
\end{bmatrix}
$$


In [4]:
X = np.array([[0, 1], 
              [1, 0]], dtype=complex)

Y = np.array([[0, -1j], 
              [1j, 0]], dtype=complex)

Z = np.array([[1, 0], 
              [0, -1]], dtype=complex)

I = np.array([[1, 0], 
              [0, 1]], dtype=complex)

Rotating $| 0 \rangle$ around the Y-axis brings us to the state $-i|1\rangle$:

In [5]:
zero_state @ Y

array([0.+0.j, 0.-1.j])

Recall the intuition that certain states are not affected by rotation, because they are already aligned with that axis. Rotating $| 0 \rangle$ or $| 1 \rangle$ around the Z-axis does not change the state. Superpositions however are affected by the Z-gate:

In [6]:
zero_state @ Z

array([1.+0.j, 0.+0.j])

The identity gate will returns the state unchanged:

In [7]:
zero_state @ I

array([1.+0.j, 0.+0.j])

# Requirements of quantum logic gates

The main requirement for a quantum logic gates is that the matrix must be unitary. This means that if multiply the matrix by its conjugate transpose, we should get the identity matrix. Recall that the conjugate transpose is just flipping the complex part of the numbers and transposing it. For any gate $U$ we have:

$$
U U^\dagger = I
$$



Let's take for example the Y-gate:

$$
Y = \begin{bmatrix}
0 & -i \\
i & 0
\end{bmatrix}
$$

$$
Y^\dagger = \begin{bmatrix}
0 & i \\
-i & 0
\end{bmatrix}
$$

$$
Y Y^\dagger = \begin{bmatrix}
0 & -i \\
i & 0
\end{bmatrix} \begin{bmatrix}
0 & i \\
-i & 0
\end{bmatrix} = \begin{bmatrix}
1 & 0 \\
0 & 1
\end{bmatrix} = I
$$


In [8]:
y_mul = Y @ Y.conj().T
y_mul

array([[1.+0.j, 0.+0.j],
       [0.+0.j, 1.+0.j]])

In [9]:
np.allclose(y_mul, I)

True

Another way to frame this requirement is that all quantum computation must be reversible. If we apply a gate we can get back the original state by applying the conjugate transpose of the gate.

In [10]:
class BaseGate(np.ndarray):
    """ Quantum gate class that checks requirements and allows us to encode/decode states."""
    def __new__(cls, input_array):
        arr = np.asarray(input_array, dtype=complex)
        obj = arr.view(cls)
        assert len(obj.shape) == 2 # Quantum gate is a 2D matrix
        assert obj.shape[0] == obj.shape[1] # Quantum gate is square
        assert np.allclose(obj @ obj.conj().T, np.eye(obj.shape[0])) # Quantum gate is unitary
        return obj
    def encodes(self, x): return np.array(self @ x, dtype=complex)
    def decodes(self, x): return np.array(self.conj().T @ x, dtype=complex)
    def __call__(self, x): return self.encodes(x)

class X(BaseGate):
    def __new__(cls):
        return super().__new__(cls, np.array([[0, 1], 
                                              [1, 0]], dtype=complex))
    
class Y(BaseGate):
    def __new__(cls):
        return super().__new__(cls, np.array([[0, -1j], 
                                              [1j, 0]], dtype=complex))

class Z(BaseGate):
    def __new__(cls):
        return super().__new__(cls, np.array([[1, 0], 
                                              [0, -1]], dtype=complex))


# TODO Explain these approach of implementing gates.

In [11]:
x_gate, y_gate, z_gate = X(), Y(), Z()
x_gate(zero_state)

array([0.+0.j, 1.+0.j])

In [12]:
x_gate.decodes(x_gate(zero_state))

array([1.+0.j, 0.+0.j])

In [13]:
class Pipeline(list):
    """ Combine multiple gates in sequence."""
    def __call__(self, x):
        for gate in self:
            x = gate.encodes(x)
        return x
    
    def decodes(self, x):
        for gate in reversed(self):
            x = gate.decodes(x)
        return x

In [14]:
pipe = Pipeline([y_gate, z_gate])

In [15]:
# Z @ Y @ |0>
output_state = pipe(zero_state)
output_state

array([0.+0.j, 0.-1.j])

In [16]:
# Decoding output yields back the original state
pipe.decodes(output_state)

array([1.+0.j, 0.+0.j])

## Clifford Group

The Clifford group are a set of common logic gates that are available on any modern quantum computer and are the fundamental building blocks for many quantum algorithms and most quantum error correction schemes focus on the Clifford group. The Pauli gates are part of the Clifford group. Additionally, the Hadamard (H) gate, S gate and CNOT gates are in the Clifford group of gates. 

# TODO Explain gates and limitations

## T-Gate

# TODO Explain how the Clifford group and T Gate allows us to approximate any state.

# Generalized Rotation

# TODO Explain General Rotation (R3) gates.

# Work In Progress (WIP)